In [3]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

In [4]:
url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo"
response = requests.get(url)

In [5]:
data = response.json()
df = pd.DataFrame(data['Time Series (5min)']).T

In [6]:
# Converting the data columns to float
df = df.astype(float)

In [7]:
df.index = pd.to_datetime(df.index)

In [8]:
df

,1. open,2. high,3. low,4. close,5. volume
2023-05-08 20:00:00,123.43,123.43,123.4300,123.430,155.0
2023-05-08 19:55:00,123.40,123.40,123.4000,123.400,860.0
2023-05-08 19:40:00,123.40,123.40,123.4000,123.400,207.0
2023-05-08 19:30:00,123.50,123.50,123.5000,123.500,300.0
2023-05-08 19:05:00,123.45,123.45,123.4500,123.450,286.0
...,...,...,...,...,...
2023-05-08 09:40:00,123.44,123.78,123.4293,123.755,35710.0
2023-05-08 09:35:00,123.76,123.92,123.2200,123.490,165793.0
2023-05-08 09:30:00,124.06,124.06,123.8500,123.850,2186.0
2023-05-08 09:15:00,124.06,124.06,124.0600,124.060,113.0


In [9]:
# We need to do this because the time is upside down in the API (lower time has to be earlier )
df = df.iloc[::-1]

In [10]:
df

,1. open,2. high,3. low,4. close,5. volume
2023-05-08 09:05:00,123.96,123.96,123.9600,123.960,303.0
2023-05-08 09:15:00,124.06,124.06,124.0600,124.060,113.0
2023-05-08 09:30:00,124.06,124.06,123.8500,123.850,2186.0
2023-05-08 09:35:00,123.76,123.92,123.2200,123.490,165793.0
2023-05-08 09:40:00,123.44,123.78,123.4293,123.755,35710.0
...,...,...,...,...,...
2023-05-08 19:05:00,123.45,123.45,123.4500,123.450,286.0
2023-05-08 19:30:00,123.50,123.50,123.5000,123.500,300.0
2023-05-08 19:40:00,123.40,123.40,123.4000,123.400,207.0
2023-05-08 19:55:00,123.40,123.40,123.4000,123.400,860.0


In [11]:
# ATR calculation
df['H-L'] = df['2. high'] - df['3. low']
df['H-PC'] = abs(df['2. high'] - df['4. close'].shift(1))
df['L-PC'] = abs(df['3. low'] - df['4. close'].shift(1))

df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
df['ATR'] = df['TR'].rolling(7).mean()

In [12]:
# Super trend indicator calculation
df['upperband'] = ((df['2. high'] + df['3. low']) / 2) + (3 * df['ATR'])
df['lowerband'] = ((df['2. high'] + df['3. low']) / 2) - (3 * df['ATR'])

df['trend'] = 0
df['trend'][0] = -1
df['supertrend'] = None
 
for i in range(0,7):
    df['trend'][i]=-1
    df['supertrend'][i] = df['upperband'][i]
for i in range(7, len(df)):
    if df['upperband'][i] < df['upperband'][i-1] or df['4. close'][i-1] > df['upperband'][i-1]:
        df['upperband'][i] = df['upperband'][i]
    else:
        df['upperband'][i] = df['upperband'][i-1]
        
    if df['lowerband'][i] > df['lowerband'][i-1] or df['4. close'][i-1] < df['lowerband'][i-1]:
        df['lowerband'][i] = df['lowerband'][i]
    else:
        df['lowerband'][i] = df['lowerband'][i-1]
    
    if df['supertrend'][i-1] == df['upperband'][i-1]:
        if df['4. close'][i] > df['upperband'][i]:
            df['trend'][i] = 1
        else:
            df['trend'][i] = -1
    else:
        if df['4. close'][i] < df['lowerband'][i]:
            df['trend'][i] = -1
        else:
            df['trend'][i] = 1
            
    if df['trend'][i] == 1:
        df['supertrend'][i] = df['lowerband'][i]
    else:
        df['supertrend'][i] = df['upperband'][i]
        

In [14]:
df

,1. open,2. high,3. low,4. close,5. volume,H-L,H-PC,L-PC,TR,ATR,upperband,lowerband,trend,supertrend
2023-05-08 09:05:00,123.96,123.96,123.9600,123.960,303.0,0.0000,NaN,NaN,0.0000,NaN,NaN,NaN,-1,NaN
2023-05-08 09:15:00,124.06,124.06,124.0600,124.060,113.0,0.0000,0.10,0.1000,0.1000,NaN,NaN,NaN,-1,NaN
2023-05-08 09:30:00,124.06,124.06,123.8500,123.850,2186.0,0.2100,0.00,0.2100,0.2100,NaN,NaN,NaN,-1,NaN
2023-05-08 09:35:00,123.76,123.92,123.2200,123.490,165793.0,0.7000,0.07,0.6300,0.7000,NaN,NaN,NaN,-1,NaN
2023-05-08 09:40:00,123.44,123.78,123.4293,123.755,35710.0,0.3507,0.29,0.0607,0.3507,NaN,NaN,NaN,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-08 19:05:00,123.45,123.45,123.4500,123.450,286.0,0.0000,0.05,0.0500,0.0500,0.021429,123.433571,123.385714,1,123.385714
2023-05-08 19:30:00,123.50,123.50,123.5000,123.500,300.0,0.0000,0.05,0.0500,0.0500,0.024286,123.572857,123.427143,1,123.427143
2023-05-08 19:40:00,123.40,123.40,123.4000,123.400,207.0,0.0000,0.10,0.1000,0.1000,0.034286,123.502857,123.427143,-1,123.502857
2023-05-08 19:55:00,123.40,123.40,123.4000,123.400,860.0,0.0000,0.00,0.0000,0.0000,0.031429,123.494286,123.305714,-1,123.494286


In [15]:
df['trend'].value_counts()

-1    68
 1    32
Name: trend, dtype: int64

Trend = -1 represents downtrend whereas Trend = 1 represents uptrend

Let us now plot upperband and lowerband and candlestick plot

In [16]:
import plotly.graph_objects as go

In [18]:
candlestick_trace =  go.Candlestick(x=df.index, open = df['1. open'],high= df['2. high'],low=df['3. low'],close=df['4. close'])
line1_trace = go.Scatter(x = df.index,y=df['upperband'],mode='lines',name='UpperBand')
line2_trace  = go.Scatter(x = df.index,y=df['lowerband'],mode='lines',name='LowerBand')
layout = go.Layout(
    title='Plot',
    xaxis_title='Time',
    yaxis_title='Stock Price',
)
fig = go.Figure(data=[candlestick_trace,line1_trace,line2_trace ], layout=layout)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()